# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [10]:
# !pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimportowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [64]:
# your code here
def generate_rsa_pair(nlen):
    assert nlen % 2 == 0

    # 1, 2
    p, q = 1, 1
    n = p * q
    while not p != q or\
          not len(bin(n)) - 2 != nlen or\
          not (nlen - 1)//2 < math.log2(p) < nlen/2 or\
              not (nlen - 1)//2 < math.log2(q) < nlen/2 or\
                not math.log2(abs(p - q)) > nlen/2 - 100 :
        p = cu.getPrime(nlen // 2)
        q = cu.getPrime(nlen // 2)
        n = p * q

    # 3
    f = (p - 1) * (q - 1)
    # 4
    # e = f - 1
    e = 65537
    while math.gcd(e, f) != 1:
        e += 2
    
    d = cu.inverse(e, f)

    return (d, n), (e, n)
    

In [65]:
generate_rsa_pair(2048)

((427801483786595073436022451938179885102909277951906509855093122731231385963021326496625711848052532565113673114359495144908426064255627783751572025674652160354945038703963027850048478755401642807957988081677026713746207239481341420404342303579930640685986124981461279536151294095035357563132128887143677010589401778151309082147133276248785341451646339291826671681242857543057902718851450233344247590067458435405954985978325338207091749706232596613395580974949212291662319404873095935174031030940625605981833570539876104137403259687389829079804210679237249320060837526547345435656571283401472776576453761836838726825,
  1593907097380448057292586892136014504263181657142359120885346104857118325290422323741293875917329097596353314092994783019435106252025075501064626256204643469765891586216124215816294892108741186168569793343312524146605183846156263369473529366101075292702528292945425120918746296822417977749573071681451686199172386040894107845571951529777604099410237750371152646613884346995517209

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [ ]:
priv_k, pub_k = generate_rsa_pair(2048)

In [112]:
def encrypt_with_rsa(M, public):
    m = cu.bytes_to_long(M.encode())
    e, n = public
    C = pow(m, e, n)
    return C


def decrypt_with_rsa(C, private):
    d, n = private
    M = pow(C, d, n)
    return cu.long_to_bytes(M).decode()



In [113]:
M = "Ala ma kota"

print(f"message: {M}")
C = encrypt_with_rsa(M, pub_k)
print(f"cipher   : {C}")
D = decrypt_with_rsa(C, priv_k)
print(f"decrypted: {D}")

message: Ala ma kota
cipher   : 8600089069227280203580822038800394529628002384245061431953025380198509347047516988054910903660760958135504771173708217708517100544553493009341342128454163699546635305194637530846311132144639412503817955728120286677738503099534151432933481364144921876731604414203227436267682540503620348533200275669598693383372020052070381922063261774964781464163445614448794917129102630416698537651569214346537020109207441381574379838326389984443561589784350383143397270908448290362031046131204683141621210625922390184419202119105414319292295346986092221339180604626236377185334528102396691157323200572545722064867561578511890585661
decrypted: Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [ ]:
# Your code here
def naive_fingerprint(M, priv_k):
    d, n = priv_k
    h = hl.sha3_256(M.encode()).hexdigest()
    h = int(h, 16)
    sig = pow(h, d, n)
    return M, sig
def naive_verifiy_fingerprint(M, sig, pub_k):
    e, n = pub_k

In [118]:
int(hl.sha3_256(M.encode()).hexdigest(), 16)

58027779095004453544284077591163918291042927990260655562844765364847025674752

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [ ]:
# Your code here

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [ ]:
M = "Ala ma kota"

# Your code here

## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [ ]:
m = "Ala ma podpis"

# Your code here

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [ ]:
# Your code here